<a href="https://colab.research.google.com/github/vibeeshK/Olympic-medals-and-elevation-levels/blob/main/project_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Hypothesis Testing: Olympic Medal Count and Elevation by Country** 

*Vibeesh Kamalakannan, Rasa Blourtchi, Raghav Arora*

## **Introduction**

"I'm a mountain guy" - Khabib Nurmagomedov. Khabib Nurmagomedov, the longest reigning UFC lightweight champion credits his physicality and athleticism to his environment, saying that living and training in the mountains, was one of the reasons for his success. This sparked the start our analysis, to see whether the environmental factor of altitute really has an impact on an athletes success. And what better way to test this theory, than on the biggest sporting event in the world... The Olympics! 

"https://www.youtube.com/watch?v=U4PW800YC-A"

## **Variables**

MedalCount: Represents the number of medals (bronze, silver and gold) won by a Country from 1896 to 2016

Average Elevation: The mean elevation (altitude) of a country 

Highest Point: The max elevation (altitude) of a country


## **Hypothesis**

As the Average Elevation or Highest Point of a country increases, the country's respective medal count in the olympics also increases. 

This is because training in higher altitudes the body triggers a hormonal response that enhances the way oxygen is delivered and utilised throughout the body. Creating more blood vessels for oxygen to flow through, altitude training may lead to improved heart functionality, enhanced muscle performance and greater overall health. With more oxygen flowing through your body, recovery times are minimised while strength and endurance is enhanced. Morevoer, studies have also suggested a positive impact on stress reduction and sleep patterns, making altitude training a workout style that offers ongoing benefits.

"https://www.surgefitness.com.au/blog/what-are-the-benefits-of-altitude-training/#:~:text=Creating%20more%20blood%20vessels%20for,strength%20and%20endurance%20is%20enhanced"


## **Libraries Used**

In [1]:
import pandas as pd 
import json
from urllib.request import urlopen # library to read data from a URL
import matplotlib.pyplot as plt # visualization library we will use that integrates with pandas
import requests 
from bs4 import BeautifulSoup
import re 
from matplotlib.pyplot import figure
import numpy as np
from scipy.stats.stats import pearsonr
import plotly.express as px
import pandas as pd
import urllib.request
import zipfile
from pandas._libs.algos import diff_2d

plt.style.use('dark_background')
figure(figsize=(8, 6), dpi=80)

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

## **Data Import and Pre-Processing**

### **Highest Point**

In [2]:
# List of highest elevation
url = "https://flagpedia.net/lists/highest-point"
data = requests.get(url).text

soup = BeautifulSoup(data, 'html.parser')
tables = soup.find_all('td')

# Create function to remove html from the extracted strings
CLEANR = re.compile('<.*?>') 
def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext

cleanhtml(str(tables[1]))
clean_values = []
for i in tables:
  clean_values.append(cleanhtml(str(i)))
clean_values[0:20]

ele=''
clean_values=[i for i in clean_values if i!=ele]
clean_values[0:20]

df = [] 
rows = [] 
for i in clean_values:
  if len(rows)==4:
    df.append(rows)
    rows = []
    rows.append(i)
  else:
    rows.append(i)
    
highest_points = pd.DataFrame(df, columns = ['Rank', 'Country','Highest Point','Height'])  
highest_points.head()  
     

,Rank,Country,Highest Point,Height
0,1.,China,Mount Everest,8 848 m
1,2.,Nepal,Mount Everest,8 848 m
2,3.,Pakistan,K2,8 611 m
3,4.,India,Kangchenjunga,8 586 m
4,5.,Bhutan,Gangkhar Puensum,7 570 m


### **Average Elevation**

In [3]:
# List of avg elevation
url = "https://www.atlasbig.com/en-us/countries-average-elevation"


data = requests.get(url).text

soup = BeautifulSoup(data, 'html.parser')
tables = soup.find_all('td')


cleanhtml(str(tables[1]))
clean_values = []
for i in tables:
  clean_values.append(cleanhtml(str(i)))
clean_values[0:20]

ele=''
clean_values=[i for i in clean_values if i!=ele]
clean_values[0:20]



df = [] 
rows = [] 
for i in clean_values:
  if len(rows)==2:
    df.append(rows)
    rows = []
    rows.append(i)
  else:
    rows.append(i)

average_elevation = pd.DataFrame(df, columns = ['Country','Average Elevation'])  
average_elevation.head()

,Country,Average Elevation
0,Bhutan,"3,280"
1,Nepal,"3,265"
2,Tajikistan,"3,186"
3,Kyrgyzstan,"2,989"
4,Lesotho,"2,161"


### **Olympic Data (1986-2016)**

In [4]:
# The URL of the zip file containing the CSV file
url = 'https://github.com/rasablourtchi/Mountain-People/blob/main/Olympic_Athletes.zip?raw=true'

# The name of the CSV file within the zip file
files_ =   ['athlete_events.csv','athlete_events_data_dictionary.csv', 'country_definitions.csv','country_definitions_data_dictionary.csv']
#csv_filename

# Download the zip file
zip_filename, _ = urllib.request.urlretrieve(url)

# Extract the CSV file from the zip file
with zipfile.ZipFile(zip_filename) as zip_file:
    with zip_file.open('athlete_events.csv') as csv_file:
        # Read the CSV file into a pandas DataFrame
        athlete_events = pd.read_csv(csv_file)

# Extract the CSV file from the zip file
with zipfile.ZipFile(zip_filename) as zip_file:
    with zip_file.open('athlete_events_data_dictionary.csv') as csv_file:
        # Read the CSV file into a pandas DataFrame
        athlete_events_data_dictionary = pd.read_csv(csv_file)

# Extract the CSV file from the zip file
with zipfile.ZipFile(zip_filename) as zip_file:
    with zip_file.open('country_definitions.csv') as csv_file:
        # Read the CSV file into a pandas DataFrame
        country_definitions = pd.read_csv(csv_file)

# Extract the CSV file from the zip file
with zipfile.ZipFile(zip_filename) as zip_file:
    with zip_file.open('country_definitions_data_dictionary.csv') as csv_file:
        # Read the CSV file into a pandas DataFrame
        country_definitions_data_dictionary = pd.read_csv(csv_file)
athlete_events.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


## **Data Cleaning and Merging** 

### **Olympic Data**

In [5]:
# Split catagorical data
athlete_events['IsBronze'] = (athlete_events['Medal'] == 'Bronze').astype(int)
athlete_events['IsSilver'] = (athlete_events['Medal'] == 'Silver').astype(int)
athlete_events['IsGold'] = (athlete_events['Medal'] == 'Gold').astype(int)

In [6]:
# Merge to get regions and NOC
olympic = pd.merge(athlete_events, country_definitions[['NOC', 'region']], on='NOC', how='left') 
olympic2 = olympic 

### **Highest Point**

In [7]:
# Clean the highest point data
for i in range(len(olympic['region'])):
    # replace hardik with shardul
    if olympic['region'][i] == 'Boliva':
        olympic['region'][i] = 'Bolivia'

    elif olympic['region'][i] == 'Czech Republic':
        olympic['region'][i] = 'Czechia'

    elif olympic['region'][i] == 'Democratic Republic of the Congo':
        olympic['region'][i] = 'DR Congo'

    elif olympic['region'][i] == 'Ivory Coast':
        olympic['region'][i] = "Côte d'Ivoire (Ivory Coast)"

    elif olympic['region'][i] == 'Macedonia':
        olympic['region'][i] = "North Macedonia"

    elif olympic['region'][i] == 'Ivory Coast':
        olympic['region'][i] = "Côte d'Ivoire (Ivory Coast)"
    
    elif olympic['region'][i] == 'Republic of Congo':
        olympic['region'][i] = "Republic of the Congo"
      
    elif olympic['region'][i] == 'Saint Kitts':
        olympic['region'][i] = "Saint Kitts and Nevis"

    elif olympic['region'][i] == 'Saint Vincent':
        olympic['region'][i] = "Saint Vincent and the Grenadines"

    elif olympic['region'][i] == 'Swaziland':
        olympic['region'][i] = "Eswatini (Swaziland)"

    elif olympic['region'][i] == 'Trinidad':
        olympic['region'][i] = "Trinidad and Tobago"

    elif olympic['region'][i] == 'UK':
        olympic['region'][i] = "United Kingdom"

    elif olympic['region'][i] == 'USA':
        olympic['region'][i] = "United States"

    elif olympic['region'][i] == 'Virgin Islands, British':
        olympic['region'][i] = "British Virgin Islands"
    
    elif olympic['region'][i] == 'Virgin Islands, US':
        olympic['region'][i] = "United States Virgin Islands"
      
olympic.head()

<ipython-input-7-fcbff94120e0>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  olympic['region'][i] = "United States"
<ipython-input-7-fcbff94120e0>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  olympic['region'][i] = "Republic of the Congo"
<ipython-input-7-fcbff94120e0>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  olympic['region'][i] = "United States Virgin Islands"
<ipython-input-7-fcbff94120e0>:14: SettingWithCopyWarnin

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,IsBronze,IsSilver,IsGold,region
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,0,0,0,China
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,0,0,0,China
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN,0,0,0,Denmark
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,0,0,1,Denmark
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN,0,0,0,Netherlands


In [8]:
# Group Medal Count by Country and Merge with Highest Elevation
clean_olympic = olympic
clean_olympic = clean_olympic.groupby('region')['IsGold', 'IsSilver', 'IsBronze'].sum()
clean_olympic['MedalCount'] = (clean_olympic['IsBronze']+clean_olympic['IsSilver']+clean_olympic['IsGold']).astype(int)
clean_olympic = clean_olympic.reset_index()
clean_olympic = clean_olympic.rename(columns={'region': 'Country'})
clean_olympic_highest = pd.merge(clean_olympic, highest_points[['Country', 'Height']], on='Country', how='left')
clean_olympic_highest = clean_olympic_highest.rename(columns={'Height': 'Highest Point'}) 
clean_olympic_highest = clean_olympic_highest[clean_olympic_highest['Highest Point'].notna()]
clean_olympic_highest['Highest Point'] = clean_olympic_highest['Highest Point'].str.replace(',', '') 
clean_olympic_highest['Highest Point'] = clean_olympic_highest['Highest Point'].str.replace('m', '') 
clean_olympic_highest['Highest Point'] = clean_olympic_highest['Highest Point'].str.replace(r'\s+', '', regex=True).astype(int)
clean_olympic_highest['Highest Point'] = pd.to_numeric(clean_olympic_highest['Highest Point']) 
clean_olympic_highest.head()

<ipython-input-8-6ae70b3a7c43>:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  clean_olympic = clean_olympic.groupby('region')['IsGold', 'IsSilver', 'IsBronze'].sum()


,Country,IsGold,IsSilver,IsBronze,MedalCount,Highest Point
0,Afghanistan,0,0,2,2,7492
1,Albania,0,0,0,0,2764
2,Algeria,5,4,8,17,3003
3,American Samoa,0,0,0,0,966
4,Andorra,0,0,0,0,2942


### **Average Elevation**

In [9]:
# Clean the avg elevation data
for i in range(len(olympic2['region'])):
    # replace hardik with shardul
    if olympic2['region'][i] == 'Boliva':
        olympic2['region'][i] = 'Bolivia'

    elif olympic2['region'][i] == 'Democratic Republic of the Congo':
        olympic2['region'][i] = 'Congo-Kinshasa'

    elif olympic2['region'][i] == 'Dominica':
        olympic2['region'][i] = 'Dominican Republic'

    elif olympic2['region'][i] == 'Ivory Coast':
        olympic2['region'][i] = "Côte d'Ivoire" 	

    elif olympic2['region'][i] == 'Netherlands':
        olympic2['region'][i] = "The Netherlands"

    elif olympic2['region'][i] == 'Republic of Congo':
        olympic2['region'][i] = "Congo-Brazzaville"
    
    elif olympic2['region'][i] == 'Russia':
        olympic2['region'][i] = "Russian Federation"

    elif olympic2['region'][i] == 'South Sudan':
        olympic2['region'][i] = "Sudan"

    elif olympic2['region'][i] == 'Trinidad':
        olympic2['region'][i] = "Trinidad and Tobago"

    elif olympic2['region'][i] == 'UK':
        olympic2['region'][i] = "United Kingdom"

    elif olympic2['region'][i] == 'United States':
        olympic2['region'][i] = "United States of America"

olympic2.head()


<ipython-input-9-df301051336e>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  olympic2['region'][i] = "The Netherlands"
<ipython-input-9-df301051336e>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  olympic2['region'][i] = "United States of America"
<ipython-input-9-df301051336e>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  olympic2['region'][i] = "Russian Federation"
<ipython-input-9-df301051336e>:11: SettingWithCopyWarning:

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,IsBronze,IsSilver,IsGold,region
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,0,0,0,China
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,0,0,0,China
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN,0,0,0,Denmark
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,0,0,1,Denmark
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN,0,0,0,The Netherlands


In [10]:
# Group Medal Count by Country and Merge with Average Elevation
clean_olympic2 = olympic2
clean_olympic2 = clean_olympic2.groupby('region')['IsGold', 'IsSilver', 'IsBronze'].sum()
clean_olympic2['MedalCount'] = (clean_olympic2['IsBronze']+clean_olympic2['IsSilver']+clean_olympic2['IsGold']).astype(int)
clean_olympic2 = clean_olympic2.reset_index()
clean_olympic2 = clean_olympic2.rename(columns={'region': 'Country'})
clean_olympic_avg = pd.merge(clean_olympic2, average_elevation[['Country', 'Average Elevation']], on='Country', how='left')
clean_olympic_avg = clean_olympic_avg[clean_olympic_avg['Average Elevation'].notna()]
clean_olympic_avg['Average Elevation'] = clean_olympic_avg['Average Elevation'].str.replace(',', '') 
clean_olympic_avg['Average Elevation'] = pd.to_numeric(clean_olympic_avg['Average Elevation'])   
clean_olympic_avg.head()

<ipython-input-10-41c83791d2af>:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  clean_olympic2 = clean_olympic2.groupby('region')['IsGold', 'IsSilver', 'IsBronze'].sum()


,Country,IsGold,IsSilver,IsBronze,MedalCount,Average Elevation
0,Afghanistan,0,0,2,2,1884
1,Albania,0,0,0,0,708
2,Algeria,5,4,8,17,800
4,Andorra,0,0,0,0,1996
5,Angola,0,0,0,0,1112


## **Data Visualization** 

### **Highest Point**

In [11]:
# Highest point per country vs medals won

fig = px.scatter(clean_olympic_highest, x='Highest Point', y='MedalCount', hover_name='Country',
                 category_orders={"Highest Point": clean_olympic_highest["Highest Point"].sort_values().unique()}) 
fig.show()

### **Average Elevation**

In [12]:
# Avg elevation vs medals

fig = px.scatter(clean_olympic_avg, x='Average Elevation', y='MedalCount', hover_name='Country',
                 category_orders={"Average Elevation": clean_olympic_avg["Average Elevation"].sort_values().unique()}) 
fig.show()

## **T Test: Pearson Coefficient** 

### **Highest Point**

In [13]:
# Pearson coefficient to find correlation between highest point and medal

pearsonr(clean_olympic_highest["Highest Point"], clean_olympic_highest["MedalCount"])


(0.17584309656065222, 0.012526842829911256)

Pearson correlation coefficient (r): 0.1758 /
Two-tailed p-value: 0.01252

Since the correlation coefficient is close to 1, this tells us that there is a no association between the highest point and medals won by the country. 

### **Average Elevation**

In [14]:
# Pearson coefficient to find correlation between avg height and medal

pearsonr(clean_olympic_avg["Average Elevation"], clean_olympic_avg["MedalCount"])


(-0.0889243353378809, 0.26499797297903244)

Pearson correlation coefficient (r):-0.08892 / 
Two-tailed p-value: 0.264997

Since the correlation coefficient is close to 1, this tells us that there is a no association between the average elevation and medals won by the country. 



## **Conclusion**

The reason we believe our analysis failed to show a relationship between Medal Count and Elevation was because we performed this analysis on a global level. By doing so,  we were unable to keep other important factors such as GDP, Education, etc constant which are known to have a stronger impact on Olympic Medal count.


## **Next Steps**



*   Conduct Analysis only on Countries with a similar socio-economic background.
*   Conduct Analysis on City and Athlete Level rather than Global Level





